In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import streamlit as st
import pickle
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [3]:
df=pd.read_csv('Churn_Modelling.csv')

In [5]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [7]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
label=LabelEncoder()
df['Gender']=label.fit_transform(df['Gender'])

In [9]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder()
geo_encoded=ohe.fit_transform(df[['Geography']])

geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [ ]:
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(), columns=ohe.get_feature_names_out(['Geography']))

In [19]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
df=pd.concat([df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [22]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
with open('label_encoder_gender.pkl','wb')as f:
    pickle.dump(label,f)


with open('ohe_geo.pkl','wb') as f:
    pickle.dump(ohe,f)  

In [12]:
X=df.drop('Exited',axis=1)
y=df['Exited']

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [14]:
st=StandardScaler()
X_train=st.fit_transform(X_train)
X_test=st.transform(X_test)

In [15]:
with open ('scaler.pkl','wb')as f:
    pickle.dump(st,f)

# ANN Implementation 


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense #Each neuron connects to all neurons in the previous layer
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
#EarlyStopping:Stops training automatically when performance stops improving
#TensorBoard:Logs training data (loss, accuracy, graphs)
import datetime

In [20]:
X_train.shape

(7500, 12)

In [39]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#1st hidden layer
    Dense(32,activation='relu'),#HL2
    Dense(1,activation='sigmoid'), #output layer
])

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 


 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [42]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [43]:
log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [44]:
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [45]:
early_stop=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [46]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stop]
)

Epoch 1/100
235/235 [==============================] - 4s 7ms/step - loss: 0.4002 - accuracy: 0.8323 - val_loss: 0.3568 - val_accuracy: 0.8548
Epoch 2/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3564 - accuracy: 0.8525 - val_loss: 0.3510 - val_accuracy: 0.8548
Epoch 3/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3469 - accuracy: 0.8557 - val_loss: 0.3546 - val_accuracy: 0.8620
Epoch 4/100
235/235 [==============================] - 1s 6ms/step - loss: 0.3425 - accuracy: 0.8603 - val_loss: 0.3514 - val_accuracy: 0.8600
Epoch 5/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3417 - accuracy: 0.8575 - val_loss: 0.3479 - val_accuracy: 0.8608
Epoch 6/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3393 - accuracy: 0.8613 - val_loss: 0.3558 - val_accuracy: 0.8596
Epoch 7/100
235/235 [==============================] - 1s 5ms/step - loss: 0.3377 - accuracy: 0.8628 - val_loss: 0.3451 - val_accuracy: 0.8640

In [47]:
model.save('model.h5')

c:\Users\BIT\Documents\deep1-learning-project\venvDeep\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [49]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 17116), started 0:11:00 ago. (Use '!kill 17116' to kill it.)